[ 신오지방 포켓몬 이미지 데이터 csv로 변환 ]

[1] 모듈 로딩 및 데이터 준비 <hr>

In [1]:
# [1-1] 모듈 로딩 및 호출 
import cv2                           # 컴퓨터비젼 모듈
import numpy as np                   # 이미지 데이터 저장 모듈
import matplotlib.pyplot as plt      # 시각화 모듈
import os                            # 파일, 폴더, 경로 관련 모듈
from rembg import remove
import cv2
import numpy as np
from PIL import Image


In [2]:
## [1-2] 데이터 준비
IMG_DIR  = './4th/'
COLOR_DIR = './color/'
FILE_CSV = './4th_color.csv'
ROW, COL = 70, 70      # 70 X 70 으로 준비 

[2] 이미지 파일 -> csv 파일 변환 

In [3]:
# [2-1] 현재 모든 이미지 파일 리스트에 저장하기 
allitems = os.listdir(IMG_DIR)

filelist = []
for item in allitems:
    # 경로 + 파일명 
    if os.path.isfile(IMG_DIR+item): 
        filelist.append(item)
        
print(filelist)

['038701.png', '038801.png', '038901.png', '039001.png', '039101.png', '039201.png', '039301.png', '039401.png', '039501.png', '039601.png', '039701.png', '039801.png', '039901.png', '040001.png', '040101.png', '040201.png', '040301.png', '040401.png', '040501.png', '040601.png', '040701.png', '040801.png', '040901.png', '041001.png', '041101.png', '041201.png', '041301.png', '041401.png', '041501.png', '041601.png', '041701.png', '041801.png', '041901.png', '042001.png', '042101.png', '042201.png', '042301.png', '042401.png', '042501.png', '042601.png', '042701.png', '042801.png', '042901.png', '043001.png', '043101.png', '043201.png', '043301.png', '043401.png', '043501.png', '043601.png', '043701.png', '043801.png', '043901.png', '044001.png', '044101.png', '044201.png', '044301.png', '044401.png', '044501.png', '044601.png', '044701.png', '044801.png', '044901.png', '045001.png', '045101.png', '045201.png', '045301.png', '045401.png', '045501.png', '045601.png', '045701.png', '0458

In [4]:
##  CSV Dataset 추가 함수 
def addDataset(csvF,label, imdata):
    # add
    data = imdata.reshape(-1)  # 1차원으로 조정 
    sdata = list(map(lambda n: str(n), data))  # 문자열로 변환
    
    # CSV 파일에 쓰기 : 라벨,p1,p2,.....
    csvF.write(str(label)+",")                 # 숫자 라벨 쓰기
    csvF.write(','.join(sdata) + '\n')         # 리스트 이미지 데이터 -> 문자열 변환 후 쓰기

In [5]:
## 전처리 함수 (사이즈 축소 : 효율성 up, 오히려 높을수록 성능 up)
def preprocessing(filename, csvF):
    # 컬러로 인식
    colorIMG=cv2.imread(IMG_DIR+filename, cv2.IMREAD_COLOR)
    
    # resize
    colorIMG =cv2.resize(colorIMG, (ROW, COL))

    # save
    ret = cv2.imwrite(f'{COLOR_DIR}g_{filename}', colorIMG)
    if not ret: print(f'{filename} - fail!')
    
    # add
    addDataset(csvF, filename[1:4], colorIMG)

In [6]:
## 이미지 데이터 증강 함수
def augmentation(filename, csvF):
    optionList = [-1, 0, 1]
    
    # 컬러로 인식
    colorIMG = cv2.imread(IMG_DIR+filename, cv2.IMREAD_COLOR)
    
    # resize
    colorIMG = cv2.resize(colorIMG, (ROW, COL))

    # flip
    for idx, option in enumerate(optionList):
        flipIMG = cv2.flip(colorIMG, option)
        addDataset(csvF, filename[1:4], flipIMG)
        N_FILE = os.path.join(COLOR_DIR, f'flip{idx}_g_{filename}')
        cv2.imwrite(N_FILE, flipIMG)

    # rotation
    H, W = colorIMG.shape[:2]  # 수정된 부분
    angles = range(0, 361, 30)
    for idx, angle in enumerate(angles):
        N_FILE = os.path.join(COLOR_DIR, f'rot{angle:03}_g_{filename}')
        M = cv2.getRotationMatrix2D((W / 2, H / 2), angle, 0.8)
        angleIMG = cv2.warpAffine(colorIMG, M, (W, H), borderValue=(255, 255, 255))  # 수정된 부분
        cv2.imwrite(N_FILE, angleIMG)
        addDataset(csvF, filename[1:4], angleIMG)


In [7]:
from rembg import remove
import cv2
import numpy as np
from PIL import Image


In [8]:
def remove_background(image):
    """
    rembg를 사용하여 이미지의 배경을 제거하는 함수
    """
    if image is None:
        return None

    # OpenCV 이미지를 PIL 이미지로 변환 (BGR → RGB)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(image_rgb)

    # 배경 제거 수행
    output_pil = remove(pil_image)

    # 다시 OpenCV 형식 (RGBA)으로 변환
    output = np.array(output_pil)
    output = cv2.cvtColor(output, cv2.COLOR_RGBA2BGRA)  # RGBA → BGRA 변환

    return output

In [9]:
def process_images(filename, csvF):
    
    img = cv2.imread(IMG_DIR + filename, cv2.IMREAD_UNCHANGED)
    
    if img is None:
        print(f"이미지를 불러올 수 없습니다: {filename}")
        return
    
    # Alpha 채널 여부 확인
    if img.shape[2] == 4:  # RGBA 포맷일 경우
        # 채널 분리
        b, g, r, a = cv2.split(img)

        # 마스크 생성 (알파값이 0보다 큰 부분)
        mask = a > 0  
        
        # 흰색 배경 이미지 생성
        img_rgb = cv2.merge((b, g, r))  # BGR 변환
        output = np.ones_like(img_rgb) * 255  # 흰색 배경
        output[mask] = img_rgb[mask]  # 객체를 흰 배경에 오버레이

    else:  # Alpha 채널이 없을 경우
        output = img

    # Resize
    colorIMG = cv2.resize(output, (ROW, COL))

    # 저장 경로 설정
    save_path = os.path.join(COLOR_DIR, f'processed_{filename}')
    ret = cv2.imwrite(save_path, colorIMG)
    
    if not ret:
        print(f"이미지 저장 실패: {save_path}")

    # 데이터셋에 추가
    addDataset(csvF, filename[1:4], colorIMG)
    
    print(f"이미지 처리 완료: {save_path}")




In [10]:
def process_and_augment_images(filename, csvF):
    import os
    import cv2
    import numpy as np

    # 이미지 로드 (Alpha 채널 포함)
    img = cv2.imread(os.path.join(IMG_DIR, filename), cv2.IMREAD_UNCHANGED)

    if img is None:
        print(f"이미지를 불러올 수 없습니다: {filename}")
        return

    # Alpha 채널 여부 확인
    if img.shape[2] == 4:  # RGBA 포맷일 경우
        # 채널 분리
        b, g, r, a = cv2.split(img)

        # 마스크 생성 (알파값이 0보다 큰 부분)
        mask = a > 0
        
        # 흰색 배경 이미지 생성
        img_rgb = cv2.merge((b, g, r))  # BGR 변환
        output = np.ones_like(img_rgb) * 255  # 흰색 배경
        output[mask] = img_rgb[mask]  # 객체를 흰 배경에 오버레이
    else:  # Alpha 채널이 없을 경우
        output = img

    # Resize (공통 처리)
    colorIMG = cv2.resize(output, (ROW, COL))

    # 저장 경로 설정 (전처리된 이미지 저장)
    save_path = os.path.join(COLOR_DIR, f'processed_{filename}')
    ret = cv2.imwrite(save_path, colorIMG)

    if not ret:
        print(f"이미지 저장 실패: {save_path}")
        return

    # 데이터셋에 추가
    addDataset(csvF, filename[1:4], colorIMG)
    print(f"이미지 처리 완료: {save_path}")

    # 증강 작업 시작
    optionList = [-1, 0, 1]

    # Flip (좌우, 상하 등 뒤집기)
    for idx, option in enumerate(optionList):
        flipIMG = cv2.flip(colorIMG, option)
        addDataset(csvF, filename[1:4], flipIMG)
        N_FILE = os.path.join(COLOR_DIR, f'flip{idx}_g_{filename}')
        cv2.imwrite(N_FILE, flipIMG)
        addDataset(csvF, filename[1:4], flipIMG)

    # Rotation (회전 증강)
    H, W = colorIMG.shape[:2]  # 이미지 높이와 너비
    angles = range(0, 361, 30)  # 0도부터 360도까지 20도 간격
    for angle in angles:
        N_FILE = os.path.join(COLOR_DIR, f'rot{angle:03}_g_{filename}')
        M = cv2.getRotationMatrix2D((W / 2, H / 2), angle, 0.8)
        angleIMG = cv2.warpAffine(colorIMG, M, (W, H), borderValue=(255, 255, 255))
        cv2.imwrite(N_FILE, angleIMG)
        addDataset(csvF, filename[1:4], angleIMG)

    # 밝기 조정
    for beta in range(-100, 101, 25):  # 밝기를 증가 또는 감소
        brightnessIMG = cv2.convertScaleAbs(colorIMG, alpha=1, beta=beta)
        N_FILE = os.path.join(COLOR_DIR, f'brightness_{beta:+}_{filename}')
        cv2.imwrite(N_FILE, brightnessIMG)
        addDataset(csvF, filename[1:4], brightnessIMG)
        
        for angle in angles:  
            rotatedIMG = cv2.warpAffine(brightnessIMG, cv2.getRotationMatrix2D((W / 2, H / 2), angle, 0.8), (W, H), borderValue=(255, 255, 255))
            rotated_file = os.path.join(COLOR_DIR, f'rot{angle:03}_brightness_{beta:+}_{filename}')
            cv2.imwrite(rotated_file, rotatedIMG)
            addDataset(csvF, filename[1:4], rotatedIMG)


    # 노이즈 추가
    noise = np.random.normal(0, 25, colorIMG.shape).astype(np.uint8)
    noisyIMG = cv2.add(colorIMG, noise)
    N_FILE = os.path.join(COLOR_DIR, f'noisy_{filename}')
    cv2.imwrite(N_FILE, noisyIMG)
    addDataset(csvF, filename[1:4], noisyIMG)

        
    for angle in angles:  
        rotatedIMG = cv2.warpAffine(noisyIMG, cv2.getRotationMatrix2D((W / 2, H / 2), angle, 0.8), (W, H), borderValue=(255, 255, 255))
        rotated_file = os.path.join(COLOR_DIR, f'rot{angle:03}_noisy_{filename}')
        cv2.imwrite(rotated_file, rotatedIMG)
        addDataset(csvF, filename[1:4], rotatedIMG)


    # 블러 추가
    kernel_sizes = [(3, 3), (5, 5), (7, 7), (9, 9), (11, 11)]  # 커널 크기
    sigmas = [0, 1, 2, 3, 5]  # 시그마 값 (0은 자동 계산)
    for i, (ksize, sigma) in enumerate(zip(kernel_sizes, sigmas)):
        # 가우시안 블러 적용
        blurIMG = cv2.GaussianBlur(colorIMG, ksize, sigma)
        
        # 파일 저장 경로 설정
        N_FILE = os.path.join(COLOR_DIR, f'blur_ksize{ksize[0]}_sigma{sigma}_{filename}')
        cv2.imwrite(N_FILE, blurIMG)
        
        for angle in angles:  
            rotatedIMG = cv2.warpAffine(blurIMG, cv2.getRotationMatrix2D((W / 2, H / 2), angle, 0.8), (W, H), borderValue=(255, 255, 255))
            rotated_file = os.path.join(COLOR_DIR, f'rot{angle:03}_blur_ksize{ksize[0]}_sigma{sigma}_{filename}')
            cv2.imwrite(rotated_file, rotatedIMG)
            addDataset(csvF, filename[1:4], rotatedIMG)


    # 다양한 확대 및 자르기 설정
    zoom_scales = [1.1, 1.2, 1.5, 1.8, 2.0]   # 확대 비율
    crop_ratios = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]  # 자를 비율

    for zoom_scale in zoom_scales:
    # 이미지 확대
        zoomIMG = cv2.resize(colorIMG, None, fx=zoom_scale, fy=zoom_scale, interpolation=cv2.INTER_LINEAR)
        H_zoom, W_zoom = zoomIMG.shape[:2]  # 확대된 이미지 크기 확인
    
    for crop_ratio in crop_ratios:
        # 자르기 시작 좌표 계산
        crop_start_x = int(W_zoom * crop_ratio)
        crop_start_y = int(H_zoom * crop_ratio)
        crop_end_x = crop_start_x + W
        crop_end_y = crop_start_y + H
        
        # 이미지 경계 확인 및 자르기
        if crop_end_x <= W_zoom and crop_end_y <= H_zoom:
            cropIMG = zoomIMG[crop_start_y:crop_end_y, crop_start_x:crop_end_x]
            N_FILE = os.path.join(COLOR_DIR, f'zoom_{zoom_scale}_crop_{crop_ratio}_{filename}')
            
            # 파일 저장 및 데이터셋에 추가
            cv2.imwrite(N_FILE, cropIMG)
            addDataset(csvF, filename[1:4], cropIMG)
            
            # 회전 추가s
            for angle in angles:  
                rotatedIMG = cv2.warpAffine(cropIMG, cv2.getRotationMatrix2D((W / 2, H / 2), angle, 0.8), (W, H), borderValue=(255, 255, 255))
                rotated_file = os.path.join(COLOR_DIR, f'rot{angle:03}_zoom_{zoom_scale}_crop_{crop_ratio}_{filename}')
                cv2.imwrite(rotated_file, rotatedIMG)
                addDataset(csvF, filename[1:4], rotatedIMG)



In [11]:
for filename in filelist:
    print(filename)
    with open(FILE_CSV, mode="a", encoding="utf-8") as csvF:
        process_and_augment_images(filename, csvF)
        

038701.png
이미지 처리 완료: ./color/processed_038701.png


038801.png
이미지 처리 완료: ./color/processed_038801.png
038901.png
이미지 처리 완료: ./color/processed_038901.png
039001.png
이미지 처리 완료: ./color/processed_039001.png
039101.png
이미지 처리 완료: ./color/processed_039101.png
039201.png
이미지 처리 완료: ./color/processed_039201.png
039301.png
이미지 처리 완료: ./color/processed_039301.png
039401.png
이미지 처리 완료: ./color/processed_039401.png
039501.png
이미지 처리 완료: ./color/processed_039501.png
039601.png
이미지 처리 완료: ./color/processed_039601.png
039701.png
이미지 처리 완료: ./color/processed_039701.png
039801.png
이미지 처리 완료: ./color/processed_039801.png
039901.png
이미지 처리 완료: ./color/processed_039901.png
040001.png
이미지 처리 완료: ./color/processed_040001.png
040101.png
이미지 처리 완료: ./color/processed_040101.png
040201.png
이미지 처리 완료: ./color/processed_040201.png
040301.png
이미지 처리 완료: ./color/processed_040301.png
040401.png
이미지 처리 완료: ./color/processed_040401.png
040501.png
이미지 처리 완료: ./color/processed_040501.png
040601.png
이미지 처리 완료: ./color/processed_040601.png
040701.png
이미지 처리 완료: ./color/p

In [12]:
## 데이터 준비2
IMG_DIR  = './dot/'
COLOR_DIR = './color/'
FILE_CSV = './4th_color.csv'
ROW, COL = 70, 70      # 70 X 70 으로 준비 

In [13]:
# [2-1] 현재 모든 이미지 파일 리스트에 저장하기 
allitems = os.listdir(IMG_DIR)

filelist = []
for item in allitems:
    # 경로 + 파일명 
    if os.path.isfile(IMG_DIR+item): 
        filelist.append(item)
        
print(filelist)

['038700.png', '038801.png', '038901.png', '039001.png', '039100.png', '039201.png', '039301.png', '039401.png', '039501.png', '039601.png', '039701.png', '039801.png', '039901.png', '040001.png', '040101.png', '040201.png', '040301.png', '040401.png', '040501.png', '040601.png', '040701.png', '040801.png', '040901.png', '041001.png', '041101.png', '041201.png', '041303.png', '041401.png', '041501.png', '041601.png', '041701.png', '041801.png', '041901.png', '042001.png', '042102.png', '042201.png', '042301.png', '042401.png', '042501.png', '042601.png', '042701.png', '042801.png', '042901.png', '043001.png', '043101.png', '043201.png', '043301.png', '043401.png', '043501.png', '043601.png', '043701.png', '043801.png', '043901.png', '044001.png', '044101.png', '044201.png', '044301.png', '044401.png', '044501.png', '044601.png', '044701.png', '044801.png', '044901.png', '045001.png', '045101.png', '045201.png', '045301.png', '045401.png', '045501.png', '045601.png', '045701.png', '0458

In [14]:
for filename in filelist:
    print(filename)
    with open(FILE_CSV, mode="a", encoding="utf-8") as csvF:
        process_and_augment_images(filename, csvF)

038700.png
이미지 처리 완료: ./color/processed_038700.png
038801.png
이미지 처리 완료: ./color/processed_038801.png
038901.png
이미지 처리 완료: ./color/processed_038901.png
039001.png
이미지 처리 완료: ./color/processed_039001.png
039100.png
이미지 처리 완료: ./color/processed_039100.png
039201.png
이미지 처리 완료: ./color/processed_039201.png
039301.png
이미지 처리 완료: ./color/processed_039301.png
039401.png
이미지 처리 완료: ./color/processed_039401.png
039501.png
이미지 처리 완료: ./color/processed_039501.png
039601.png
이미지 처리 완료: ./color/processed_039601.png
039701.png
이미지 처리 완료: ./color/processed_039701.png
039801.png
이미지 처리 완료: ./color/processed_039801.png
039901.png
이미지 처리 완료: ./color/processed_039901.png
040001.png
이미지 처리 완료: ./color/processed_040001.png
040101.png
이미지 처리 완료: ./color/processed_040101.png
040201.png
이미지 처리 완료: ./color/processed_040201.png
040301.png
이미지 처리 완료: ./color/processed_040301.png
040401.png
이미지 처리 완료: ./color/processed_040401.png
040501.png
이미지 처리 완료: ./color/processed_040501.png
040601.png
이미지 처리 완료: ./color/p